<center><h1><b>Python: Neural Network Implementation </b></h1></center>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import Input
from sklearn.model_selection import train_test_split

%matplotlib inline

In [2]:
data = {'age': [22,25, 47, 52, 46, 56, 40, 55, 60, 62, 61, 18, 28, 27, 29, 49, 55, 25, 58, 19, 18, 21, 26, 40, 45, 50, 54, 23],
       'affordability': [1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1],
       'bought_insurance': [0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0]}

In [3]:
df = pd.DataFrame(data)
df.head()

,age,affordability,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [4]:
df.shape

(28, 3)

In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,28.0,39.678571,15.451366,18.0,25.0,42.5,54.25,62.0
affordability,28.0,0.642857,0.487950,0.0,0.0,1.0,1.00,1.0
bought_insurance,28.0,0.500000,0.509175,0.0,0.0,0.5,1.00,1.0


In [6]:
X_train, X_test, y_train, y_test = train_test_split(df[['age', 'affordability']], df['bought_insurance'], test_size=0.2, random_state=0)

In [12]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [7]:
def sigmoid(x):
    import math
    return 1 / (1 + math.exp(-x))

In [8]:
def prediction_function(age, affordability):
    weighted_sum = coef[0] * age + coef[1] * affordability + intercept
    return sigmoid(weighted_sum)

In [9]:
# Putting the log loss in a function

def log_loss(y_true, y_predicted):
    import numpy as np
    epsilon = 1e-15
    # Clip the predicted values between epsilon and 1 - epsilon
    y_predicted_new = np.clip(y_predicted, epsilon, 1 - epsilon)
    # Log loss calculation
    return -np.mean(y_true * np.log(y_predicted_new) + (1 - y_true) * np.log(1 - y_predicted_new))

In [10]:
def sigmoid_numpy(x):
    return 1/(1 + np.exp(-x))

In [44]:
class myNN:
    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.bias = 0
    
    def fit(self, X, y, epochs, loss_threshold):
        self.w1, self.w2, self.bias = self.gradient_descent(X['age'], X['affordability'], y, epochs, loss_threshold)
    
    def predict(self, X_test):
        # Predicting output
        weighted_sum = self.w1 * X_test['age'] + self.w2 * X_test['affordability'] + self.bias
        y_predicted = sigmoid_numpy(weighted_sum)
        # Convert sigmoid output to binary values (0 or 1)
        return np.where(y_predicted >= 0.5, 1, 0)
    
    def gradient_descent(self, age, affordability, y_true, epochs, loss_threshold):
        w1 = w2 = 1
        bias = 0
        rate = 0.5
        n = len(age)

        for i in range(epochs):
            # Forward pass
            weighted_sum = w1 * age + w2 * affordability + bias
            y_predicted = sigmoid_numpy(weighted_sum)

            # Compute loss (log loss)
            loss = log_loss(y_true, y_predicted)

            # Compute accuracy
            accuracy = self.calculate_accuracy(y_true, np.where(y_predicted >= 0.5, 1, 0))

            # Compute gradients
            w1d = (1/n) * np.dot(np.transpose(age), (y_predicted - y_true))
            w2d = (1/n) * np.dot(np.transpose(affordability), (y_predicted - y_true))
            bias_d = np.mean(y_predicted - y_true)

            # Update weights and bias
            w1 = w1 - rate * w1d
            w2 = w2 - rate * w2d
            bias = bias - rate * bias_d
            
            # Print every 50 epochs or when the loss is below the threshold
            if i % 50 == 0 or loss <= loss_threshold:
                print(f'Epoch: {i}, w1: {w1}, w2: {w2}, bias: {bias}, loss: {loss}, accuracy: {accuracy}')
            
            # Stop training if loss goes below the threshold
            if loss <= loss_threshold:
                break

        # Return final weights and bias
        return w1, w2, bias

    # Method to calculate accuracy
    def calculate_accuracy(self, y_true, y_predicted):
        accuracy = np.mean(y_true == y_predicted) * 100
        return accuracy

    # Method to evaluate the model on test data
    def evaluate(self, X_test, y_test):
        # Predict the test data
        y_pred = self.predict(X_test)
        
        # Calculate accuracy
        accuracy = self.calculate_accuracy(y_test, y_pred)
        
        # Calculate loss
        weighted_sum = self.w1 * X_test['age'] + self.w2 * X_test['affordability'] + self.bias
        y_predicted_proba = sigmoid_numpy(weighted_sum)  # Predicted probabilities for log loss
        loss = log_loss(y_test, y_predicted_proba)
        
        print(f'Evaluation - Loss: {loss}, Accuracy: {accuracy}')
        return loss, accuracy

In [45]:
customModel = myNN()
customModel.fit(X_train_scaled, y_train, epochs=500, loss_threshold = 0.2925)

Epoch: 0, w1: 0.976400033722807, w2: 0.9530287297658705, bias: -0.11929523268087014, loss: 0.6904170412461691, accuracy: 50.0
Epoch: 50, w1: 1.6273282403886429, w2: 1.3952047623196655, bias: -1.5077580472348355, loss: 0.5064613660815618, accuracy: 72.72727272727273
Epoch: 100, w1: 2.4776463122344032, w2: 1.7530783340850946, bias: -2.1497001037712895, loss: 0.45531603351033323, accuracy: 72.72727272727273
Epoch: 150, w1: 3.2923040619449124, w2: 1.9504980745355582, bias: -2.648289202150659, loss: 0.41705968009482874, accuracy: 81.81818181818183
Epoch: 200, w1: 4.05533761036354, w2: 2.0798864813030113, bias: -3.06886968285289, loss: 0.3859105055861656, accuracy: 90.9090909090909
Epoch: 250, w1: 4.764199191501739, w2: 2.178884337335988, bias: -3.4409180085410727, loss: 0.35980062022328674, accuracy: 90.9090909090909
Epoch: 300, w1: 5.42151831634678, w2: 2.263935548308929, bias: -3.7792871018742615, loss: 0.33758577351842683, accuracy: 90.9090909090909
Epoch: 350, w1: 6.031788551457507, w2:

In [46]:
customModel.predict(X_test_scaled)

array([1, 0, 0, 0, 1, 0])

In [41]:
y_test

2     1
22    0
14    0
17    1
5     1
11    0
Name: bought_insurance, dtype: int64

In [47]:
customModel.evaluate(X_test_scaled, y_test)


Evaluation - Loss: 0.6338137592267873, Accuracy: 83.33333333333334


(0.6338137592267873, 83.33333333333334)